In [5]:
### automatically refresh the buffer

%load_ext autoreload
%autoreload 2

### solve the auto-complete issue

%config Completer.use_jedi = False

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import warnings
warnings.filterwarnings('ignore')
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Wedge, Circle
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
regions_ = [ 'CT', 'DE', 'ME', 'MD', 'MA', 'NH', 'NJ', 'NY', 'PA', 
             'RI', 'VT', 'WV', 'IA', 'MI', 'MN', 'WI', 'IL', 'IN', 
             'MO', 'OH', 'AL', 'FL', 'GA', 'NC', 'SC', 'VA', 'TN', 
             'KY', 'AR', 'LA', 'MS', 'MT', 'NE', 'ND', 'SD', 'WY', 
             'KS', 'OK', 'TX', 'AZ', 'CO', 'NM', 'UT', 'CA', 'NV',
             'ID', 'OR', 'WA',]

In [4]:
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/'

# 设置起始年份和结束年份
start_year = 1979
end_year = start_year+2
for year in range(start_year, end_year):  # 1989不包含
    # 处理年份和月份范围
    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)

    # 遍历月份
    for month in months:
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
         
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)

            for state in regions_:
                print("Current Time:", datetime.now().time().strftime("%H:%M:%S"))
                state_gdf = gdf[gdf['STUSPS'] == state]
                output_folder = '../CONUS404_data/' + state + '/'
                output_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
                output_path = os.path.join(output_folder, output_file)
                os.makedirs(output_folder, exist_ok=True)
                shp_s = state_gdf
                lon = ds['XLONG'].values
                lat = ds['XLAT'].values
                grid = gpd.GeoDataFrame(
                    geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                    index=np.arange(lon.size)
                )
                grid.set_crs(shp_s.crs, inplace=True)
                grid_s = gpd.sjoin(grid, shp_s, how='inner', predicate='within')

                mask = np.full(ds['PREC_ACC_NC'].shape[1:], False)  # 初始化为全 False
                for index in grid_s.index:
                    row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                    mask[row, col] = True
                mask_da = xr.DataArray(mask, dims=ds['PREC_ACC_NC'].dims[1:], coords={'south_north': ds['PREC_ACC_NC'].coords['south_north'], 'west_east': ds['PREC_ACC_NC'].coords['west_east']})
                ds_s = ds.where(mask_da, drop=True)
                ds_s.to_netcdf(output_folder + output_file)

Current Time: 18:17:02
Current Time: 18:17:04
Current Time: 18:17:06
Current Time: 18:17:08


KeyboardInterrupt: 

In [8]:
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/'

# 设置起始年份和结束年份
start_year = 1979
end_year = start_year+2
for year in range(start_year, end_year):  # 1989不包含
    # 处理年份和月份范围
    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)

    # 遍历月份
    for month in months:
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
         
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)

            for state in regions_:
                print("Current Time:", datetime.now().time().strftime("%H:%M:%S"))
                state_gdf = gdf[gdf['STUSPS'] == state]
                output_folder = 'testCONUS404DATA/' + state + '/'
                output_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
                output_path = os.path.join(output_folder, output_file)
                os.makedirs(output_folder, exist_ok=True)
                shp_s = state_gdf
                lon = ds['XLONG'].values
                lat = ds['XLAT'].values
                grid = gpd.GeoDataFrame(
                    geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                    index=np.arange(lon.size)
                )
                grid.set_crs(shp_s.crs, inplace=True)
                grid_s = gpd.sjoin(grid, shp_s, how='inner', predicate='within')

                mask = np.full(ds['PREC_ACC_NC'].shape[1:], False)  # 初始化为全 False
                for index in grid_s.index:
                    row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                    mask[row, col] = True
                mask_da = xr.DataArray(mask, dims=ds['PREC_ACC_NC'].dims[1:], coords={'south_north': ds['PREC_ACC_NC'].coords['south_north'], 'west_east': ds['PREC_ACC_NC'].coords['west_east']})
                ds_s = ds.where(mask_da, drop=True)
                ds_s.to_netcdf(output_folder + output_file)

Current Time: 22:47:21
Current Time: 22:47:23
Current Time: 22:47:24
Current Time: 22:47:27
Current Time: 22:47:29
Current Time: 22:47:31
Current Time: 22:47:33
Current Time: 22:47:35
Current Time: 22:47:37
Current Time: 22:47:39
Current Time: 22:47:41
Current Time: 22:47:43
Current Time: 22:47:45
Current Time: 22:47:48
Current Time: 22:47:50
Current Time: 22:47:52
Current Time: 22:47:55
Current Time: 22:47:57
Current Time: 22:47:59
Current Time: 22:48:02
Current Time: 22:48:04
Current Time: 22:48:06
Current Time: 22:48:08
Current Time: 22:48:10
Current Time: 22:48:12
Current Time: 22:48:15
Current Time: 22:48:17
Current Time: 22:48:19
Current Time: 22:48:21
Current Time: 22:48:23
Current Time: 22:48:25
Current Time: 22:48:28
Current Time: 22:48:30
Current Time: 22:48:32
Current Time: 22:48:34
Current Time: 22:48:37
Current Time: 22:48:39
Current Time: 22:48:41
Current Time: 22:48:43
Current Time: 22:48:46
Current Time: 22:48:48
Current Time: 22:48:50
Current Time: 22:48:52
Current Tim

KeyboardInterrupt: 